In [ ]:
# Dating Cleaning and Merging
# Anita Sun
# River Wang

In [12]:
import numpy as np
import geopandas as gpd
import pandas as pd
from preprocess import clean_police_dataset
import shapely
from shapely.geometry import point

In [51]:
gdf = gpd.read_file('illinois_shapefiles/tl_2023_17_tract.shp')

In [52]:
gdf.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,17,161,022800,17161022800,1400000US17161022800,228,Census Tract 228,G5020,S,2103943,0,+41.4991066,-090.5472913,"POLYGON ((-90.55724 41.49433, -90.55724 41.494..."
1,17,161,022900,17161022900,1400000US17161022900,229,Census Tract 229,G5020,S,1311568,0,+41.4897868,-090.5477824,"POLYGON ((-90.55731 41.49337, -90.55727 41.493..."
2,17,161,023000,17161023000,1400000US17161023000,230,Census Tract 230,G5020,S,4680875,127822,+41.4757633,-090.5435406,"POLYGON ((-90.55779 41.46279, -90.55772 41.463..."
3,17,161,023100,17161023100,1400000US17161023100,231,Census Tract 231,G5020,S,3496624,66115,+41.4684205,-090.5721252,"POLYGON ((-90.5826 41.46694, -90.5826 41.46714..."
4,17,019,010901,17019010901,1400000US17019010901,109.01,Census Tract 109.01,G5020,S,140475117,118829,+39.9889767,-088.2028522,"POLYGON ((-88.27831 39.88075, -88.27786 39.883..."


In [53]:
pol_df = clean_police_dataset('police_data.csv')

In [54]:
pol_df['latitude'].dtype

dtype('float64')

In [55]:
pol_df['geometry'] = gpd.points_from_xy(pol_df['longitude'], pol_df['latitude'])
pol_gdf = gpd.GeoDataFrame(pol_df, geometry='geometry', crs="EPSG:4326")

In [56]:
pol_gdf.head()

,name,age,gender,race,date,street_address,city,state,zip,county,...,pop_native_american_census_tract,pop_asian_census_tract,pop_pacific_islander_census_tract,pop_other_multiple_census_tract,pop_hispanic_census_tract,lat_long,month,day,year,geometry
0,Steven Espinoza,36.0,Male,Hispanic,2025-01-12,N Mountain Ave and 11th St,upland,CA,91786,San Bernardino,...,NaN,NaN,NaN,NaN,NaN,"(34.1033261, -117.6701864)",1,12,2025,POINT (-117.67019 34.10333)
1,Jose Evans,42.0,Male,Hispanic,2025-01-12,8500 block of Cermak Rd,north riverside,IL,60546,Cook,...,NaN,NaN,NaN,NaN,NaN,"(41.850411, -87.8351933)",1,12,2025,POINT (-87.83519 41.85041)
2,"Benjamin Prowell, Jr.",34.0,Male,Black,2025-01-11,10000 block of Crystal Hill Rd,maumelle,AR,72113,Pulaski,...,NaN,NaN,NaN,NaN,NaN,"(34.8185095, -92.3715459)",1,11,2025,POINT (-92.37155 34.81851)
3,Brian Rolstad,43.0,Male,Unknown race,2025-01-11,900 block of W 23rd St,los angeles,CA,90007,Los Angeles,...,NaN,NaN,NaN,NaN,NaN,"(34.0333907, -118.2800794)",1,11,2025,POINT (-118.28008 34.03339)
4,Devin Shields,23.0,Male,Unknown race,2025-01-11,2300 block of Waverly Dr,gary,IN,46404,Lake,...,NaN,NaN,NaN,NaN,NaN,"(41.5961595, -87.366104)",1,11,2025,POINT (-87.3661 41.59616)


In [61]:
geo_joined_df = gpd.sjoin(gdf.to_crs(crs='EPSG:26916'), pol_gdf.to_crs(crs='EPSG:26916'))

In [62]:
geo_joined_df.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'GEOIDFQ', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'geometry', 'index_right', 'name', 'age', 'gender', 'race',
       'date', 'street_address', 'city', 'state', 'zip', 'county',
       'agency_responsible', 'ori', 'cause_of_death', 'circumstances',
       'disposition_official', 'officer_charged', 'news_urls',
       'signs_of_mental_illness', 'allegedly_armed', 'wapo_armed',
       'wapo_threat_level', 'wapo_flee', 'geography', 'encounter_type',
       'initial_reason', 'call_for_service', 'tract',
       'hhincome_median_census_tract', 'latitude', 'longitude',
       'pop_total_census_tract', 'pop_white_census_tract',
       'pop_black_census_tract', 'pop_native_american_census_tract',
       'pop_asian_census_tract', 'pop_pacific_islander_census_tract',
       'pop_other_multiple_census_tract', 'pop_hispanic_census_tract',
       'lat_long', 'month', 'day', 'year'],
      dtyp

In [63]:
geo_joined_df = geo_joined_df[['GEOIDFQ','name', 'age', 'gender', 'race',
       'date', 'street_address', 'city', 'state', 'zip', 'county',
       'agency_responsible', 'ori', 'cause_of_death', 'circumstances',
       'disposition_official', 'officer_charged', 'news_urls',
       'signs_of_mental_illness', 'allegedly_armed', 'wapo_armed',
       'wapo_threat_level', 'wapo_flee', 'geography', 'encounter_type',
       'initial_reason', 'call_for_service', 'tract',
       'hhincome_median_census_tract', 'latitude', 'longitude',
       'pop_total_census_tract', 'pop_white_census_tract',
       'pop_black_census_tract', 'pop_native_american_census_tract',
       'pop_asian_census_tract', 'pop_pacific_islander_census_tract',
       'pop_other_multiple_census_tract', 'pop_hispanic_census_tract',
       'lat_long', 'month', 'day', 'year']]

In [64]:
geo_joined_df.rename(columns={'GEOIDFQ': 'GEO_ID'}, inplace=True)

In [65]:
geo_joined_df.head()

,GEO_ID,name,age,gender,race,date,street_address,city,state,zip,...,pop_black_census_tract,pop_native_american_census_tract,pop_asian_census_tract,pop_pacific_islander_census_tract,pop_other_multiple_census_tract,pop_hispanic_census_tract,lat_long,month,day,year
7,1400000US17019010702,Randy Allen Jackson,39.0,Male,Black,2023-07-04,2200 County Road 2250 East,st joseph,IL,61873,...,2%,0%,2%,0%,0%,1%,"(40.0871982, -88.0334101)",7,4,2023
12,1400000US17199021001,Joshua Green,27.0,Male,White,2015-04-28,1408 W Main St,marion,IL,62959,...,11%,0%,2%,0%,3%,3%,"(37.731275, -88.941288)",4,28,2015
18,1400000US17119403531,Kody C. Waters,31.0,Male,Unknown race,2021-05-22,819 Edwardsville Rd.,troy,IL,62294,...,1%,0%,2%,0%,2%,5%,"(38.732431, -89.904604)",5,22,2021
23,1400000US17119400951,Kyeiree V. Myers,28.0,Male,Black,2023-08-23,Highway 162 at Mockingbird Lane,granite city,IL,62040,...,2%,0%,2%,0%,5%,3%,"(38.7087957, -90.0942946)",8,23,2023
31,1400000US17119403403,Kevin C. Steinhauer,45.0,Male,White,2022-03-02,100 block of Arnold Street,collinsville,IL,62234,...,21%,0%,2%,0%,2%,11%,"(38.66012, -90.02783)",3,2,2022


In [66]:
census_demo_df = pd.read_csv('merged_result.csv')

/tmp/ipykernel_177875/3180693781.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,182,183,184,185,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,216,217,218,219,220,221,222,223,224,225,226,227,228,230,231,232,233,236,238,239,240,241,244,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,

In [67]:
census_demo_df= census_demo_df.drop(index=0, axis=0)

In [68]:
col_list = census_demo_df.columns

In [69]:
col_list = list(col_list[2:-1])

In [70]:
for col in col_list:
    try:
        census_demo_df[col] = census_demo_df[col].astype(float)
    except Exception as e:
        continue

In [71]:
census_demo_df.head()

,GEO_ID,NAME_demo,DP05_0001E,DP05_0001M,DP05_0002E,DP05_0002M,DP05_0003E,DP05_0003M,DP05_0004E,DP05_0004M,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
1,1400000US17001000100,"Census Tract 1, Adams County, Illinois",4607.0,407.0,2064.0,243.0,2543.0,300.0,81.2,12.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1400000US17001000201,"Census Tract 2.01, Adams County, Illinois",2003.0,326.0,905.0,134.0,1098.0,267.0,82.4,21.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1400000US17001000202,"Census Tract 2.02, Adams County, Illinois",2391.0,300.0,1143.0,184.0,1248.0,217.0,91.6,20.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1400000US17001000400,"Census Tract 4, Adams County, Illinois",3314.0,545.0,1680.0,376.0,1634.0,333.0,102.8,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,1400000US17001000500,"Census Tract 5, Adams County, Illinois",2154.0,350.0,992.0,181.0,1162.0,211.0,85.4,14.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [74]:
geo_joined_df['GEO_ID'].dtype

dtype('O')

In [79]:
df_illinois_merged = pd.merge(geo_joined_df, census_demo_df, on='GEO_ID')

In [80]:
df_illinois_merged.head()

,GEO_ID,name,age,gender,race,date,street_address,city,state,zip,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
0,1400000US17019010702,Randy Allen Jackson,39.0,Male,Black,2023-07-04,2200 County Road 2250 East,st joseph,IL,61873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1400000US17199021001,Joshua Green,27.0,Male,White,2015-04-28,1408 W Main St,marion,IL,62959,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1400000US17119403531,Kody C. Waters,31.0,Male,Unknown race,2021-05-22,819 Edwardsville Rd.,troy,IL,62294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1400000US17119400951,Kyeiree V. Myers,28.0,Male,Black,2023-08-23,Highway 162 at Mockingbird Lane,granite city,IL,62040,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1400000US17119403403,Kevin C. Steinhauer,45.0,Male,White,2022-03-02,100 block of Arnold Street,collinsville,IL,62234,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [81]:
df_illinois_merged.to_csv('merged_data_illinois.csv')

In [82]:
ny_gdf = gpd.read_file('new_york_shapefiles/tl_2023_36_tract.shp')

In [83]:
ny_joined_df = gpd.sjoin(ny_gdf.to_crs(crs='EPSG:26916'), pol_gdf.to_crs(crs='EPSG:26916'))

In [85]:
ny_joined_df = ny_joined_df[['GEOIDFQ','name', 'age', 'gender', 'race',
       'date', 'street_address', 'city', 'state', 'zip', 'county',
       'agency_responsible', 'ori', 'cause_of_death', 'circumstances',
       'disposition_official', 'officer_charged', 'news_urls',
       'signs_of_mental_illness', 'allegedly_armed', 'wapo_armed',
       'wapo_threat_level', 'wapo_flee', 'geography', 'encounter_type',
       'initial_reason', 'call_for_service', 'tract',
       'hhincome_median_census_tract', 'latitude', 'longitude',
       'pop_total_census_tract', 'pop_white_census_tract',
       'pop_black_census_tract', 'pop_native_american_census_tract',
       'pop_asian_census_tract', 'pop_pacific_islander_census_tract',
       'pop_other_multiple_census_tract', 'pop_hispanic_census_tract',
       'lat_long', 'month', 'day', 'year']]

In [86]:
ny_joined_df.rename(columns={'GEOIDFQ': 'GEO_ID'}, inplace=True)

In [91]:
ny_census_df = pd.read_csv('merged_result_newyork.csv')

/tmp/ipykernel_177875/1186665232.py:1: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,228,236,244,304,318,352,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,44

In [94]:
ny_census_df = ny_census_df.drop(index=0, axis=0)

In [95]:
col_list = list(ny_census_df.columns[2:-1])

In [96]:
for col in col_list:
    try:
        ny_census_df[col] = ny_census_df[col].astype(float)
    except Exception as e:
        continue

In [99]:
df_new_york_merged = pd.merge(ny_joined_df, ny_census_df, on="GEO_ID")

In [100]:
df_new_york_merged.head()

,GEO_ID,name,age,gender,race,date,street_address,city,state,zip,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
0,1400000US36065021104,Nyah Mway,13.0,Male,Asian,2024-06-28,900 block of Shaw Street,utica,NY,13502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1400000US36065021104,Shatelle Hooks,35.0,Male,Black,2019-01-07,1303 Lincoln Ave,utica,NY,13502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1400000US36081097204,Quayshawn Samuel,31.0,Male,Black,2022-12-29,439 Beach 56th,queens,NY,11692,...,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1400000US36055009605,Mark Gaskill,28.0,Male,White,2021-05-14,Glasser St. and Masseth St.,rochester,NY,14606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1400000US36081099200,Zabina Gafoor,52.0,Female,Asian,2023-02-17,Beach Channel Drive and Bay 32nd Street,queens,NY,11691,...,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [101]:
df_new_york_merged.to_csv('merged_data_new_york.csv')